__This notebook__ explains how to fine-tune GPT-2 Large on a large dataset in colab or on your home computer.

To fit this task into a colab instance, we will use two tricks:
* streaming the [C4 dataset](https://huggingface.co/datasets/c4) using [`datasets` Streaming API](https://huggingface.co/docs/datasets/dataset_streaming.html). Without that, C4 would take up over 300GB of disk space.
* training with 8-Bit Adam from the [`bitsandbytes` library](https://github.com/facebookresearch/bitsandbytes). Without 8-bit compression, training GPT-2 Large would not fit in GPU memory.



This notebook is based on the ["fine-tune a language model"](https://github.com/huggingface/notebooks/blob/master/examples/language_modeling.ipynb) tutorial by [Sylvain Gugger](https://sgugger.github.io/pages/about-me.html#about-me) as well as the [pytorch language-model example](https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_clm_no_trainer.py).

# Installation

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [1]:
#  ! pip install datasets transformers



We are also installing bitsandbytes which depends on the CUDA version run by your colab. The installed CUDA version is displayed in the top right when calling nvidia-smi. Use this version to install the right bitsandbytes version below. We need a GPU for this, so if you have not yet a GPU loaded use Runtime-> Change runtime type -> select GPU from the dropdown.

In [2]:
# ! nvidia-smi
# ! pip install bitsandbytes-cuda112


To test the bitsandbytes installation we can run a simple update with 8-bit Adam.

In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"


In [2]:
import bitsandbytes as bnb
import torch

p = torch.nn.Parameter(torch.rand(10, 10).cuda())
a = torch.rand(10, 10).cuda()

p1 = p.data.sum().item()

adam = bnb.optim.Adam8bit([p])

out = a * p
loss = out.sum()
loss.backward()
adam.step()

p2 = p.data.sum().item()

assert p1 != p2
print("SUCCESS!")
print("Installation was successful!")


/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/kosenko/miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /home/kosenko/miniconda3/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/kosenko/miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...
SUCCESS!
Installation was successful!


# Dataset Streaming

Pre-training often requires a huge text dataset. Some famous datasets for pre-training are [C4](https://huggingface.co/datasets/c4), its multilingual version [mC4](https://huggingface.co/datasets/mc4), as well as [OSCAR](https://huggingface.co/datasets/oscar).

These datasets can be terabytes of data and require a lot of resources:
- a good bandwidth to download all the data
- terabytes of disk space to store the data
- dozens of CPUs and a good infrastructure to tokenize the text dataset
- lots of time to wait for the tokenization to happen

This makes it very impractical to get your hands on a dataset for pretraining when you have limited resources.

Dataset streaming is the solution in this case. Streaming allows to simply have access to the very small subset of the dataset that you need at any time during training. Text samples are progressively downloaded during training, and processed on-the-fly.

Thanks to dataset streaming:
- training can start directly without waiting for terabytes of data to be downloaded
- you can use an arbitrarily large dataset, without being constrained by your disk space
- you can process the batches of text as they arrive with a regular CPU
- you don't waste time processing examples that are not immediately needed for training

Dataset streaming is available in [Hugging Face Datasets](https://github.com/huggingface/datasets) - you simply need to pass `streaming=True` when loading a dataset, and it can be used with a PyTorch data loader.

![dataset streaming](https://huggingface.co/docs/datasets/_images/stream.gif "Dataset Streaming")

Hugging face Datasets also allows you to process examples on-the-fly via `.map()` and shuffle the dataset with `.shuffle()`.

Here is an example on how to load the C4 dataset in streaming mode:

In [3]:
from datasets import load_dataset

c4 = load_dataset("c4", "en", streaming=True)

# Let's print a few examples
for i, example in enumerate(c4["train"]):
    print(f"{i}: {str(example)[:200]}...")
    if i == 5:
        break


0: {'text': 'Beginners BBQ Class Taking Place in Missoula!\nDo you want to get better at making delicious BBQ? You will have the opportunity, put this on your calendar now. Thursday, September 22nd join ...
1: {'text': 'Discussion in \'Mac OS X Lion (10.7)\' started by axboi87, Jan 20, 2012.\nI\'ve got a 500gb internal drive and a 240gb SSD.\nWhen trying to restore using disk utility i\'m given the error "N...
2: {'text': 'Foil plaid lycra and spandex shortall with metallic slinky insets. Attached metallic elastic belt with O-ring. Headband included. Great hip hop or jazz dance costume. Made in the USA.', 'tim...
3: {'text': "How many backlinks per day for new site?\nDiscussion in 'Black Hat SEO' started by Omoplata, Dec 3, 2010.\n1) for a newly created site, what's the max # backlinks per day I should do to be s...
4: {'text': 'The Denver Board of Education opened the 2017-18 school year with an update on projects that include new construction, upgrades, heat mitigation and quality le

# Fine-tuning a language model

We are fine-tuning without Trainer so we can specify dataset streaming and 8-bit optimizers. For that, we copy the main parts of the [example script for language modeling](https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_clm_no_trainer.py). First we import boiler-plate and the go step-by-step throught the model setup and training.

## Preprocessing & model setup

In [4]:
# You can also adapt this script on your own causal language modeling task. Pointers for this are left as comments.

import argparse
import logging
import math
import os
import random
from itertools import chain
from pathlib import Path

import datasets
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

import transformers
from huggingface_hub import Repository
from transformers import (
    CONFIG_MAPPING,
    MODEL_MAPPING,
    AdamW,
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    SchedulerType,
    default_data_collator,
    get_scheduler,
    set_seed,
)
from transformers.file_utils import get_full_repo_name
from transformers.utils.versions import require_version


logger = logging.getLogger(__name__)

require_version(
    "datasets>=1.16.1",
    "To fix: pip install -r examples/pytorch/language-modeling/requirements.txt",
)

MODEL_CONFIG_CLASSES = list(MODEL_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)


def parse_args():
    parser = argparse.ArgumentParser(
        description="Finetune a transformers model on a causal language modeling task"
    )
    parser.add_argument(
        "--dataset_name",
        type=str,
        default=None,
        help="The name of the dataset to use (via the datasets library).",
    )
    parser.add_argument(
        "--dataset_config_name",
        type=str,
        default=None,
        help="The configuration name of the dataset to use (via the datasets library).",
    )
    parser.add_argument(
        "--text_column_name",
        type=str,
        default=None,
        help="The name of the column containing the text data.",
    )
    parser.add_argument(
        "--dataset_streaming",
        action="store_true",
        help="If passed, will use dataset streaming (via the datasets library)",
    )
    parser.add_argument(
        "--model_name_or_path",
        type=str,
        help="Path to pretrained model or model identifier from huggingface.co/models.",
        required=False,
    )
    parser.add_argument(
        "--config_name",
        type=str,
        default=None,
        help="Pretrained config name or path if not the same as model_name",
    )
    parser.add_argument(
        "--tokenizer_name",
        type=str,
        default=None,
        help="Pretrained tokenizer name or path if not the same as model_name",
    )
    parser.add_argument(
        "--use_slow_tokenizer",
        action="store_true",
        help="If passed, will use a slow tokenizer (not backed by the 🤗 Tokenizers library).",
    )
    parser.add_argument(
        "--per_device_train_batch_size",
        type=int,
        default=1,
        help="Batch size (per device) for the training dataloader.",
    )
    parser.add_argument(
        "--learning_rate",
        type=float,
        default=5e-5,
        help="Initial learning rate (after the potential warmup period) to use.",
    )
    parser.add_argument(
        "--weight_decay", type=float, default=0.0, help="Weight decay to use."
    )
    parser.add_argument(
        "--num_train_epochs",
        type=int,
        default=1,
        help="Total number of training epochs to perform.",
    )
    parser.add_argument(
        "--max_train_steps",
        type=int,
        default=None,
        help="Total number of training steps to perform. If provided, overrides num_train_epochs.",
    )
    parser.add_argument(
        "--gradient_accumulation_steps",
        type=int,
        default=1,
        help="Number of updates steps to accumulate before performing a backward/update pass.",
    )
    parser.add_argument(
        "--lr_scheduler_type",
        type=SchedulerType,
        default="linear",
        help="The scheduler type to use.",
        choices=[
            "linear",
            "cosine",
            "cosine_with_restarts",
            "polynomial",
            "constant",
            "constant_with_warmup",
        ],
    )
    parser.add_argument(
        "--num_warmup_steps",
        type=int,
        default=3000,
        help="Number of steps for the warmup in the lr scheduler.",
    )
    parser.add_argument(
        "--output_dir", type=str, default=None, help="Where to store the final model."
    )
    parser.add_argument(
        "--seed", type=int, default=None, help="A seed for reproducible training."
    )
    parser.add_argument(
        "--model_type",
        type=str,
        default=None,
        help="Model type to use if training from scratch.",
        choices=MODEL_TYPES,
    )
    parser.add_argument(
        "--block_size",
        type=int,
        default=None,
        help="Optional input sequence length after tokenization. The training dataset will be truncated in block of this size for training. Default to the model max input length for single sentence inputs (take into account special tokens).",
    )
    parser.add_argument(
        "--preprocessing_num_workers",
        type=int,
        default=None,
        help="The number of processes to use for the preprocessing.",
    )
    parser.add_argument(
        "--overwrite_cache",
        type=bool,
        default=False,
        help="Overwrite the cached training and evaluation sets",
    )
    parser.add_argument(
        "--no_keep_linebreaks",
        action="store_true",
        help="Do not keep line breaks when using TXT files.",
    )
    parser.add_argument(
        "--push_to_hub",
        action="store_true",
        help="Whether or not to push the model to the Hub.",
    )
    parser.add_argument(
        "--hub_model_id",
        type=str,
        help="The name of the repository to keep in sync with the local `output_dir`.",
    )
    parser.add_argument(
        "--hub_token", type=str, help="The token to use to push to the Model Hub."
    )
    args = parser.parse_args(args=[])

    if args.push_to_hub:
        assert (
            args.output_dir is not None
        ), "Need an `output_dir` to create a repo when `--push_to_hub` is passed."

    return args


We setup the streaming dataset, the tokenizer, and the model (GPT-2 medium).

In [5]:
args = parse_args()  # get default arguments

# If passed along, set the training seed now.
if args.seed is not None:
    set_seed(args.seed)

args.dataset_name = "c4"
args.dataset_streaming = True
args.text_column_name = "text"
args.model_name_or_path = "gpt2-large"
args.dataset_config_name = "en"
args.block_size = 1024
args.max_train_steps = 1_000_000
args.log_loss_interval = 25


# LOAD DATA
raw_train_dataset = load_dataset(
    args.dataset_name,
    args.dataset_config_name,
    streaming=args.dataset_streaming,
    split="train",
)

if args.config_name:
    config = AutoConfig.from_pretrained(args.config_name)
elif args.model_name_or_path:
    config = AutoConfig.from_pretrained(args.model_name_or_path)
else:
    config = CONFIG_MAPPING[args.model_type]()
    logger.warning("You are instantiating a new config instance from scratch.")

# TOKENIZER
if args.tokenizer_name:
    tokenizer = AutoTokenizer.from_pretrained(
        args.tokenizer_name, use_fast=not args.use_slow_tokenizer
    )
elif args.model_name_or_path:
    tokenizer = AutoTokenizer.from_pretrained(
        args.model_name_or_path, use_fast=not args.use_slow_tokenizer
    )
else:
    raise ValueError(
        "You are instantiating a new tokenizer from scratch. This is not supported by this script."
        "You can do it from another script, save it, and load it from here, using --tokenizer_name."
    )

if args.model_name_or_path:
    model = AutoModelForCausalLM.from_pretrained(
        args.model_name_or_path,
        from_tf=bool(".ckpt" in args.model_name_or_path),
        config=config,
    )
else:
    logger.info("Training new model from scratch")
    model = AutoModelForCausalLM.from_config(config)

model.resize_token_embeddings(len(tokenizer))

model.gradient_checkpointing_enable()
model.cuda()  # send model to cuda preemptively to free RAM. Yep, we're using GPU as an offload memory. O tempora! O mores!
None


We now preprocess the dataset into blocks of the specified size (768).

In [19]:
text_column_name = args.text_column_name


def tokenize_function(examples):
    # print(tokenizer(examples[text_column_name]))
    return tokenizer(
        examples[text_column_name],
        truncation=True,
        max_length=1024,
    )


tokenized_train_dataset = raw_train_dataset.shuffle(10_000).map(tokenize_function)

# Note that with `batched=True`, this map processes 1,000 texts together, so group_texts throws away a remainder
# for each of those groups of 1,000 texts. You can adjust that batch_size here but a higher value might be slower
# to preprocess.
train_dataset = tokenized_train_dataset
train_dataset = train_dataset.shuffle(
    10_000,
).with_format("torch")

# DataLoaders creation:
train_dataloader = DataLoader(
    train_dataset, collate_fn=default_data_collator, batch_size=2
)


In [ ]:
next(iter(train_dataset))["input_ids"]


## 8-bit Optimizers


In this example, we fine-tune GPT-2 medium with a sequence dimension of 768 which runs out of memory. How can we fit this model on a colab GPU with 12 GB of memory? One solution is to use 8-bit optimizers.

8-bit opitimizers decrease the memory footprint for training models by compressing and storing the optimizer statistics for optimizers. For Adam, there are two optimizer buffers, one for an estimate of the running mean of the gradient and one for the standard deviation. Each of the buffers has the size of the full model, as such, the Adam optimizers uses 2x more memory than the model itself. With 8-bit optimizers we reduce that from 32-bit to 8-bit thus reducing the memory due to Adam from 2x the model size to 0.5x the model size -- a reduction by 75%.

8-bit optimizers work by using dynamic quantization and block-wise quantization to ensure stable training and the same performance as 32-bit optimizers while achieving the 75% reduction in memory.

8-bit optimizers work as follows
1. Chunk optimizer states into blocks
2. Normalize each block into the range [-1, 1] by dividing by the absmax of the block
3. Perform dynamic quantization
4. Store 8-bit data

For dequantization we reverse these steps. These steps are demonstrated by the example below:

 ![Schematic of 8-bit optimizers](https://timdettmers.com/wp-content/uploads/2021/12/schematic2.svg)



In [7]:
import bitsandbytes as bnb

# optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate) # this crashes with out-of-memory error
optimizer = bnb.optim.Adam8bit(model.parameters(), lr=args.learning_rate)


## Train the model

Now we are training the model with dataset streaming and 8-bit optimizers.

In [9]:
lr_scheduler = get_scheduler(
    name=args.lr_scheduler_type,
    optimizer=optimizer,
    num_warmup_steps=args.num_warmup_steps,
    num_training_steps=args.max_train_steps,
)
# Train!
total_batch_size = args.per_device_train_batch_size * args.gradient_accumulation_steps
# Only show the progress bar once on each machine.
progress_bar = tqdm(range(args.max_train_steps), disable=False)
completed_steps = 0


def get_free_mem():
    t = torch.cuda.get_device_properties(0).total_memory
    r = torch.cuda.memory_reserved(0)
    a = torch.cuda.memory_allocated(0)
    f = r - a
    return f / 1024**3, r / 1024**3, a / 1024**3


for epoch in range(args.num_train_epochs):
    model.train()
    losses = []
    for step, batch in enumerate(train_dataloader):
        print(batch)
        gpu_data = {}
        for key, value in batch.items():
            gpu_data[key] = value.cuda()

        outputs = model(**gpu_data, use_cache=False)
        loss = outputs.loss
        losses.append(loss.item())
        loss = loss / args.gradient_accumulation_steps
        loss.backward()
        if step % args.gradient_accumulation_steps == 0 or step == args.max_train_steps:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)
            completed_steps += 1

        if step % args.log_loss_interval == 0 and step > 0:
            try:
                perplexity = math.exp(sum(losses) / len(losses))
            except OverflowError:
                perplexity = float("inf")
            losses = []
            print(f"epoch: {epoch+1}, step: {step}, perplexity: {perplexity}")

        if completed_steps >= args.max_train_steps:
            break


  0%|          | 0/1000000 [02:36<?, ?it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:23                                                                                   │
│                                                                                                  │
│   20 for epoch in range(args.num_train_epochs):                                                  │
│   21 │   model.train()                                                                           │
│   22 │   losses = []                                                                             │
│ ❱ 23 │   for step, batch in enumerate(train_dataloader):                                         │
│   24 │   │   print(batch)                                                                        │
│   25 │   │   gpu_data = {}                                                                       │
│   26 │   │   for key, value in batch.items():                                                    │
│                                                                                                  │
│ /home/kosenko/miniconda3/lib/python3.10/site-packages/torch/utils/data/dataloader.py:628 in      │
│ __next__                                                                                         │
│                                                                                                  │
│    625 │   │   │   if self._sampler_iter is None:                                                │
│    626 │   │   │   │   # TODO(https://github.com/pytorch/pytorch/issues/76750)                   │
│    627 │   │   │   │   self._reset()  # type: ignore[call-arg]                                   │
│ ❱  628 │   │   │   data = self._next_data()                                                      │
│    629 │   │   │   self._num_yielded += 1                                                        │
│    630 │   │   │   if self._dataset_kind == _DatasetKind.Iterable and \                          │
│    631 │   │   │   │   │   self._IterableDataset_len_called is not None and \                    │
│                                                                                                  │
│ /home/kosenko/miniconda3/lib/python3.10/site-packages/torch/utils/data/dataloader.py:671 in      │
│ _next_data                                                                                       │
│                                                                                                  │
│    668 │                                                                                         │
│    669 │   def _next_data(self):                                                                 │
│    670 │   │   index = self._next_index()  # may raise StopIteration                             │
│ ❱  671 │   │   data = self._dataset_fetcher.fetch(index)  # may raise StopIteration              │
│    672 │   │   if self._pin_memory:                                                              │
│    673 │   │   │   data = _utils.pin_memory.pin_memory(data, self._pin_memory_device)            │
│    674 │   │   return data                                                                       │
│                                                                                                  │
│ /home/kosenko/miniconda3/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py:44 in     │
│ fetch                                                                                            │
│                                                                                                  │
│   41 │   │   │   │   raise StopIteration                                                         │
│   42 │   │   else:                                                                               │
│   43 │   │   │   data = next(self.dataset_iter)                                                  │
│ ❱ 44 │   │   return self.collate_fn(data)                  